In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

In [ ]:
labels = ['O', 'R']
img_size = 64
def get_data(data_dir):
    data = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
            resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
            data.append([resized_arr, class_num])
    return np.array(data)

In [ ]:
train = get_data('TRAIN')
val = get_data('VALIDATION')
test = get_data('TEST')

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []
for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)

for feature, label in test:
    x_test.append(feature)
    y_test.append(label)

# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255
x_test = np.array(x_test) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

x_test.reshape(-1, img_size, img_size, 1)
y_test = np.array(y_test)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

In [ ]:
dropout = 0.4
#dropout = 0.5
#dropout = 0.3

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(64,64,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())

model.add(Conv2D(64, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(dropout))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax"))

model.summary()

In [ ]:
learning = 0.000001
#learning = 0.000002
#learning = 0.000005
#learning = 0.00001

In [ ]:
opt = Adam(lr=learning)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train,y_train,epochs = 5 , validation_data = (x_val, y_val))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict_classes(x_val)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_val, predictions, target_names = ['Organic (Class 0)','Recyclable (Class 1)']))

In [ ]:
test_predictions = model.predict_classes(x_test)
test_predictions = test_predictions.reshape(1,-1)[0]
print(classification_report(y_test, test_predictions, target_names = ['Organic (Class 0)','Recyclable (Class 1)']))

In [ ]:
#components = 30
#components = 20
components = 10

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(components)
compressed_images = []
for i in range(len(x_train)):
    img = x_train[i]
    red,green,blue = cv2.split(img)
    red_transformed = pca.fit_transform(red)
    red_inverted = pca.inverse_transform(red_transformed)

    #Applying to Green channel and then applying inverse transform to transformed array.
    green_transformed = pca.fit_transform(green)
    green_inverted = pca.inverse_transform(green_transformed)

    #Applying to Blue channel and then applying inverse transform to transformed array.
    blue_transformed = pca.fit_transform(blue)
    blue_inverted = pca.inverse_transform(blue_transformed)

    img_compressed = (np.dstack((red_inverted, green_inverted, blue_inverted))) 
    compressed_images.append(img_compressed)

In [ ]:
compressed_train = np.array(compressed_images)

In [ ]:
compressed_images = []
for i in range(len(x_val)):
    img = x_val[i]
    red,green,blue = cv2.split(img)
    red_transformed = pca.fit_transform(red)
    red_inverted = pca.inverse_transform(red_transformed)

    #Applying to Green channel and then applying inverse transform to transformed array.
    green_transformed = pca.fit_transform(green)
    green_inverted = pca.inverse_transform(green_transformed)

    #Applying to Blue channel and then applying inverse transform to transformed array.
    blue_transformed = pca.fit_transform(blue)
    blue_inverted = pca.inverse_transform(blue_transformed)

    img_compressed = (np.dstack((red_inverted, green_inverted, blue_inverted))) 
    compressed_images.append(img_compressed)

In [ ]:
compressed_val = np.array(compressed_images)

In [ ]:
compressed_images = []
for i in range(len(x_test)):
    img = x_val[i]
    red,green,blue = cv2.split(img)
    red_transformed = pca.fit_transform(red)
    red_inverted = pca.inverse_transform(red_transformed)

    #Applying to Green channel and then applying inverse transform to transformed array.
    green_transformed = pca.fit_transform(green)
    green_inverted = pca.inverse_transform(green_transformed)

    #Applying to Blue channel and then applying inverse transform to transformed array.
    blue_transformed = pca.fit_transform(blue)
    blue_inverted = pca.inverse_transform(blue_transformed)

    img_compressed = (np.dstack((red_inverted, green_inverted, blue_inverted))) 
    compressed_images.append(img_compressed)

In [ ]:
compressed_test = np.array(compressed_images)

In [ ]:
datagen.fit(compressed_train)

In [ ]:
history = model.fit(compressed_train,y_train,epochs = 5 , validation_data = (compressed_val, y_val))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(5)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
predictions = model.predict_classes(compressed_val)
predictions = predictions.reshape(1,-1)[0]
print(classification_report(y_val, predictions, target_names = ['Organic (Class 0)','Recyclable (Class 1)']))

In [ ]:
predictions_test = model.predict_classes(compressed_test)
predictions_test = predictions_test.reshape(1,-1)[0]
print(classification_report(y_test, predictions_test, target_names = ['Organic (Class 0)','Recyclable (Class 1)']))

In [ ]:
# FOr clustering
var = x_train
var = var.reshape(len(var),-1)
pca = PCA(10)
var = pca.fit_transform(var)
np.sum(pca.explained_variance_ratio_)

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(2, random_state=0)
labels = kmeans.fit(var).predict(var)
print(np.sum(labels==y_train))
print(np.sum(labels!=y_train))
print(np.sum(labels == 0))
print(np.sum(labels==1))
print(np.sum(y_train==0))
print(np.sum(y_train==1))

In [ ]:
var = x_train
var = var.reshape(len(var),-1)
pca = PCA(20)
var = pca.fit_transform(var)
np.sum(pca.explained_variance_ratio_)

In [ ]:
labels = kmeans.fit(var).predict(var)
print(np.sum(labels==y_train))
print(np.sum(labels!=y_train))
print(np.sum(labels == 0))
print(np.sum(labels==1))
print(np.sum(y_train==0))
print(np.sum(y_train==1))

In [ ]:
var = x_train
var = var.reshape(len(var),-1)
pca = PCA(30)
var = pca.fit_transform(var)
np.sum(pca.explained_variance_ratio_)

In [ ]:
labels = kmeans.fit(var).predict(var)
print(np.sum(labels==y_train))
print(np.sum(labels!=y_train))
print(np.sum(labels == 0))
print(np.sum(labels==1))
print(np.sum(y_train==0))
print(np.sum(y_train==1))

In [ ]:
var = x_train
var = var.reshape(len(var),-1)
pca = PCA(1000)
var = pca.fit_transform(var)
np.sum(pca.explained_variance_ratio_)

In [ ]:
labels = kmeans.fit(var).predict(var)
print(np.sum(labels==y_train))
print(np.sum(labels!=y_train))
print(np.sum(labels == 0))
print(np.sum(labels==1))
print(np.sum(y_train==0))
print(np.sum(y_train==1))